In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
from keras.layers import GlobalAveragePooling2D
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier=Sequential()
#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3)))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))
#卷積組合
classifier.add(Convolution2D(32,(5,5),input_shape=(32,32,3)))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
classifier.add(MaxPooling2D(pool_size=(3,3)))
#flatten
##classifier.add(Flatten())
classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(100)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100,verbose = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 11s 218us/step - loss: 1.5981 - accuracy: 0.4266
Epoch 2/100
50000/50000 [==============================] - 4s 89us/step - loss: 1.3566 - accuracy: 0.5267
Epoch 3/100
50000/50000 [==============================] - 4s 87us/step - loss: 1.2679 - accuracy: 0.5599
Epoch 4/100
50000/50000 [==============================] - 4s 86us/step - loss: 1.2205 - accuracy: 0.5774
Epoch 5/100
50000/50000 [==============================] - 4s 86us/step - loss: 1.1853 - accuracy: 0.5905
Epoch 6/100
50000/50000 [==============================] - 4s 86us/step - loss: 1.1492 - accuracy: 0.6056
Epoch 7/100
50000/50000 [==============================] - 4s 87us/step - loss: 1.1240 - accuracy: 0.6119
Epoch 8/100
50000/50000 [==============================] - 4s 87us/step - loss: 1.1028 - accuracy: 0.6196
Epoch 9/100
50000/50000 [==============================] - 4s 87us/step - loss: 1.0861 - accuracy: 0.6267
Epoch 10/100
50000/50000 [==================

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.00675505, 0.00726382, 0.08186439, 0.10548773, 0.39933294,
        0.03040555, 0.347094  , 0.01354788, 0.00141565, 0.00683301]],
      dtype=float32)